# Introduction

In this project, our objective is to process and analyze a dataset of security alerts in order to extract meaningful insights and build a solid foundation for modeling......

It is often said that data science is composed of approximately 40% statistics, 40% computer science, and the remaining 20% business understanding. This final component is essential: before we attempt to build machine learning models or artificial intelligence systems, we must first understand the business domain — in this case, cybersecurity.
A strong understanding of the context allows us to make informed decisions, identify relevant patterns, and potentially uncover valuable insights that could significantly improve our models.

For that reason, we begin this project by examining and understanding our data in detail, including a thorough explanation of each column. Once the context is clear, we proceed to the exploratory data analysis.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)  


In [3]:
path = r"C:\Users\F1302\Desktop\AlgoSecure\security_alerts_dataset.csv"
df = pd.read_csv(path, sep=",", encoding="utf-8", low_memory=False)
df.head(3)

,alert_id,timestamp,alert_type,severity,source_ip,destination_ip,affected_application,description,technical_indicator,status,event_count,user_involved,detection_tool,raw_output,cvss_score,cve_id,ports
0,ALERT-21ca6360,2025-03-19 08:42:15.688764,Malware Detection,High,45.76.12.145,10.0.0.21,Email Server,Port scan from 45.76.12.145 detected 8 open ports on 10.0.0.21,e67f227ab078285c12917af979dbbe32,New,2,NaN,Rustscan,"{\n ""host"": ""10.0.0.21"",\n ""ports"": [\n 21,\n 25,\n 1433,\n 80,\n 27017,\n 3306,\n 9200,\n 8443\n ],\n ""open_ports"": 8,\n ""scan_time"": ""122.52ms"",\n ""services"": [\n ""21/FTP"",\n ""25/SMTP"",\n ""1433/MSSQL"",\n ""80/HTTP"",\n ""27017/MongoDB"",\n ""3306/MySQL"",\n ""9200/Elasticsearch"",\n ""8443/HTTPS-Alt""\n ],\n ""rustscan_version"": ""2.1.1"",\n ""timestamp"": ""2025-04-02T08:41:01.782567""\n}",NaN,NaN,"21,25,1433,80,27017,3306,9200,8443"
1,ALERT-6c9ac310,2025-03-19 08:42:58.688764,DoS/DDoS,Low,89.23.45.181,10.0.0.184,Internal Portal,DoS/DDoS event detected from 89.23.45.181,b0b214938a9b7a13228b6f227676c28f,New,5,user398@company.com,EDR,NaN,NaN,NaN,NaN
2,ALERT-fdc937b2,2025-03-19 08:43:11.688764,Policy Violation,Low,45.76.12.162,172.16.1.55,VPN Service,Nuclei scan detected Jenkins Detect vulnerability on 172.16.1.55,36cc676b1c9d36aa727d3f06f59021e0,False Positive,1,user652@company.com,Nuclei,"{\n ""template"": ""jenkins-detect"",\n ""matcher-name"": ""jenkins-detect-detect"",\n ""severity"": ""low"",\n ""host"": ""172.16.1.55"",\n ""matched-at"": ""https://172.16.1.55/api/endpoint"",\n ""extracted-results"": [\n ""version: vulnerable-2.x""\n ],\n ""ip"": ""172.16.1.55"",\n ""timestamp"": ""2025-04-02T08:41:01.753512"",\n ""curl-command"": ""curl -X GET -H 'User-Agent: Mozilla/5.0' https://172.16.1.55/api/endpoint"",\n ""cvss-score"": ""0.8"",\n ""cve"": ""CVE-2020-8099""\n}",0.846068,CVE-2020-8099,443


By viewing the head:
Alert_id: looks like the primary key of the table (we will verify if all values are uniques)
Timestamp: Maybe is the date of the alert 
 

# Understunding the data

In [4]:
print(df.columns)
print(f"Shape of data: {df.shape}")

Index(['alert_id', 'timestamp', 'alert_type', 'severity', 'source_ip',
       'destination_ip', 'affected_application', 'description',
       'technical_indicator', 'status', 'event_count', 'user_involved',
       'detection_tool', 'raw_output', 'cvss_score', 'cve_id', 'ports'],
      dtype='object')
Shape of data: (5252, 17)


## Dataset Column Descriptions

This document provides a clear and concise explanation of each column in the security alert dataset.

---

**alert_id**  
Unique identifier for each alert. Used to reference or track a specific alert.

**timestamp**  
Date and time when the alert was generated by the detection tool.

**alert_type**  
Type of detected event, such as malware detection, phishing attempt, denial-of-service attack, etc.

**severity**  
The severity level assigned to the alert:  
- `Low`: Low risk  
- `Medium`: Moderate risk  
- `High`: High risk

**source_ip**  
The IP address from which the alert originated. Usually represents the potential attacker or external source.

**destination_ip**  
The internal IP address targeted by the alert — typically a device or service within the organization.

**affected_application**  
The name of the application, service, or system affected by the alert (e.g., email server, database, VPN service).

**description**  
Automatically generated text describing the detected event. It may be generic or semi-dynamic depending on the tool.

**technical_indicator**  
Technical fingerprint identifying the threat (e.g., a hash, signature, or rule ID).  
This may be duplicated across rows if the **same threat** is detected multiple times in different contexts.

**status**  
The current processing state of the alert:  
- `New`: Not yet reviewed  
- `Resolved`: Investigated and closed  
- `False Positive`: Considered irrelevant or harmless

**event_count**  
The number of similar events grouped into this single alert. Used to reflect how frequently or intensely the event occurred in a short time.

**user_involved**  
The internal user account (e.g., email) that was involved in or targeted by the alert. Usually represents the victim.

**detection_tool**  
The security tool or system that generated the alert (e.g., SIEM, EDR, Nuclei, Rustscan).

**raw_output**  
The unprocessed output from the detection tool, usually in JSON or log format. It contains technical details that may not be fully extracted into other columns.

**cvss_score**  
A standardized score (0 to 10) indicating the severity of a known vulnerability, based on the CVSS (Common Vulnerability Scoring System).

**cve_id**  
A public identifier for a documented vulnerability (in the format CVE-YYYY-XXXX). This allows linking to public vulnerability databases (e.g., CVE-2020-8099).

**ports**  
List of network ports associated with the alert. These may have been scanned, exploited, or simply observed in suspicious traffic.


In [5]:
df.info()
# From the output, we can say that, the timestamp column is in object format, we need to convert it to datetime format.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5252 entries, 0 to 5251
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   alert_id              5252 non-null   object 
 1   timestamp             5252 non-null   object 
 2   alert_type            5252 non-null   object 
 3   severity              5252 non-null   object 
 4   source_ip             5252 non-null   object 
 5   destination_ip        5252 non-null   object 
 6   affected_application  5252 non-null   object 
 7   description           5252 non-null   object 
 8   technical_indicator   5252 non-null   object 
 9   status                5252 non-null   object 
 10  event_count           5252 non-null   int64  
 11  user_involved         1421 non-null   object 
 12  detection_tool        5252 non-null   object 
 13  raw_output            941 non-null    object 
 14  cvss_score            1789 non-null   float64
 15  cve_id               

In [6]:
df.alert_id.unique().shape[0] == df.shape[0]
# this means that alert_id is unique for each row

True

In [7]:
df.isna().sum()/df.shape[0]*100

alert_id                 0.000000
timestamp                0.000000
alert_type               0.000000
severity                 0.000000
source_ip                0.000000
destination_ip           0.000000
affected_application     0.000000
description              0.000000
technical_indicator      0.000000
status                   0.000000
event_count              0.000000
user_involved           72.943641
detection_tool           0.000000
raw_output              82.083016
cvss_score              65.936786
cve_id                  75.418888
ports                   49.904798
dtype: float64

In [8]:
df.duplicated().sum()

0

Based on the outputs of `df.info()` and the missing value analysis, we observe the following:

- **`alert_id`** is unique for each row, confirming that each alert corresponds to a distinct observation.
- There are **no duplicated rows** in the dataset (`df.duplicated().sum() == 0`), which means that all rows are unique.
- The **`timestamp`** column represents the date and time when the alert was generated. However, its current type is `object`. It should be converted to `datetime` for proper temporal analysis.

### Missing Data Summary

Several columns contain a high percentage of missing values:

- **`user_involved`**: 73% missing
- **`cvss_score`**: 66% missing
- **`cve_id`**: 75% missing
- **`ports`**: 50% missing
- **`raw_output`**: 82% missing


These variables require further investigation to determine:

1. Whether the missing values are informative (e.g., not all alerts involve users or known vulnerabilities),
2. If imputation or default values are relevant and valid,
3. Or whether some variables should be dropped due to low utility.

We will now explore these columns in detail before making any cleaning decisions.


In [9]:
# timestamp to datetime
df.timestamp = pd.to_datetime(df.timestamp, format="%Y-%m-%d %H:%M:%S.%f")

In [10]:
([df.user_involved.value_counts()[df.user_involved.value_counts()==i].shape[0] for i in range(1, 8)])

[275, 237, 103, 61, 20, 2, 1]

### Analysis of `user_involved`

The column `user_involved` contains values for only 27% of the observations (i.e., 1,418 non-null entries out of 5,252).
To better understand the distribution of these values, we computed the frequency of how many times each user appears. The breakdown is as follows:

- **275** users appear only **once**
- **237** users appear **twice**
- **103** users appear **three times**
- **61** users appear **four times**
- **20** users appear **five times**
- **2** users appear **six times**
- **1** user appears **seven times**

#### Insights

- A large portion of the `user_involved` values are **repeated**, suggesting that some users are involved in multiple alerts.
- However, more than 19% of the `user_involved` entries (i.e., 275 out of 1,418) are **unique**, which may represent isolated or low-risk incidents.
- The presence of repeated users might indicate:
  - Targeted attacks toward specific accounts
  - Misconfigurations or vulnerabilities consistently linked to the same users
  - Compromised accounts being exploited repeatedly

This pattern can be useful when scoring or prioritizing alerts:
- Frequent user involvement may suggest higher urgency.
- Unique or one-time users might be deprioritized unless the alert severity is high.

We may further analyze the relationship between `user_involved` and other fields such as `severity`, `alert_type`, and `status`.
#### On the Usefulness of `user_attack_count` Feature

One potential idea is to create a feature such as `user_attack_count`, which would represent the total number of alerts in which a given user appears. This could help identify users who are repeatedly targeted and potentially prioritize them for investigation.

However, there is a major limitation:

- The original `user_involved` column is **73% missing**, meaning that this user-level information is available in **only 27%** of the dataset.
- As a result, any derived feature like `user_attack_count` would also be **missing in at least 73% of rows**.

#### Conclusion

While the idea of tracking how many times a user is involved in alerts is valid and potentially useful, the **sparsity of the original variable** makes it unreliable as a feature for general modeling or scoring purposes. It might still provide insights for a **subset analysis** (e.g., among rows where `user_involved` is present), **but it cannot be used as a global predictive feature without introducing strong bias.**


### Analysis of `cvss_score`

#### What is a CVSS Score and Why It Matters

**CVSS** stands for **Common Vulnerability Scoring System**. It is an open standard used to measure the severity of a known security vulnerability (CVE). The score is calculated based on several technical factors such as:

- How easily the vulnerability can be exploited
- Whether it requires user interaction
- What kind of impact it has on confidentiality, integrity, or availability
- The level of privileges required to exploit it

The score ranges from **0.0 to 10.0**, and is commonly categorized as:

| Score Range | Severity Level |
|-------------|----------------|
| 0.0         | None           |
| 0.1 – 3.9   | Low            |
| 4.0 – 6.9   | Medium         |
| 7.0 – 8.9   | High           |
| 9.0 – 10.0  | Critical       |

##### Relationship with `cve_id`

A **`cve_id`** (Common Vulnerabilities and Exposures identifier) refers to a publicly known and documented vulnerability.  
Each `cve_id` is typically associated with **one official CVSS score**, published and maintained in vulnerability databases such as the **NVD (National Vulnerability Database)**.

In other words:
- **CVE = the name/identifier of a vulnerability**
- **CVSS = the severity score assigned to that CVE**

This relationship allows security teams to evaluate how critical a known vulnerability is, and to act accordingly.

> If a security alert is linked to a `cve_id`, retrieving the official CVSS score can help assess the real technical risk behind that alert.

##### Why is the CVSS Score Useful?

- It provides a **standardized way to evaluate risk**, across different organizations and tools.
- It helps cybersecurity teams **prioritize which vulnerabilities to address first**, especially when dealing with hundreds or thousands of alerts.
- It can be used to **automate alert scoring or triage** in detection systems.
- It supports compliance with security standards (e.g., ISO 27001, NIST).

However, it is important to note that the CVSS score only tells **how dangerous a vulnerability is in theory** — it does **not confirm** whether the alert is a true positive or a false alarm. Contextual analysis is still needed.


#### How We Handle the `cvss_score`

The `cvss_score` is a severity score associated with a known vulnerability identified by a `cve_id`. In theory, **if we have a `cve_id`**, we can retrieve its official `cvss_score` from public databases such as the [NVD (National Vulnerability Database)](https://nvd.nist.gov).

However, the inverse is **not always true**:  
> A `cvss_score` can exist **without a `cve_id`**, especially when the alert is based on a generic detection rule, custom scoring, or behavioral analysis rather than a formally registered vulnerability.  
> In such cases, it's impossible to trace the score back to an official CVE — and the score may be approximate or even heuristic.

---

#### Reliability of the Scores in This Dataset

After investigation and direct confirmation from the challenge organizer, we know that:
- The `cvss_score` in this dataset was **automatically generated by a detection tool**
- It is **neither official nor necessarily accurate**
- It was included **purely for the purpose of this challenge**

---

#### Strategy and Justification

Given that, we make the following decision:

> **If a `cve_id` is present**, we will **replace the existing `cvss_score`** with the **official score retrieved from the NVD**.  
> **If no `cve_id` is present but a `cvss_score` exists**, we will **keep the existing score**, assuming it was generated in a plausible way.

##### Why this approach?

- Many `cve_id` values in the dataset do not return any `cvss_score` from the NVD (some are invalid or incomplete), leading to **over 90% missing scores** if we rely only on official sources.
- In a real-world context, we would have access to more complete and validated threat intelligence sources to enrich missing data.
- Given the nature of this dataset (synthetically generated for a recruitment challenge), **we allow ourselves this treatment** in order to demonstrate a thoughtful and practical approach to data enrichment.

---

## What Would We Do in a Real Dataset?

If this were a production-grade dataset in a real company or SOC (Security Operations Center), we would:

- **Only use official CVSS scores** pulled from reliable sources (NVD, vendor advisories, threat intelligence platforms)
- **Discard or flag scores that do not have a traceable `cve_id`**
- **Never train models on unverified or synthetic data**, especially for risk prioritization

We would also consider:
- Using **IP reputation feeds** (e.g., AbuseIPDB)
- Cross-referencing with MITRE ATT&CK or CVE impact scores
- Building internal scoring rules based on past incident response data

---

#### ⚠️ Important Disclaimer

> In a real-world cybersecurity context, we **would never merge official and non-official scores without clearly separating them**.  
> Trust and traceability are fundamental. Using unreliable data to prioritize threats could **lead to real damage** — such as ignoring a critical attack or reacting to a false alarm.

That's why in production environments, **we always prioritize data quality and source integrity** over coverage or volume.


In [ ]:
import requests
import time
from tqdm import tqdm

# Fonction pour récupérer le CVSS officiel à partir d'un cve_id via l'API de la NVD
def get_cvss_from_nvd(cve_id, api_key=None):
    """
    Query the NVD (National Vulnerability Database) API to retrieve the official CVSS score
    for a given CVE ID. Tries CVSS v3.1 first, and falls back to CVSS v2 if not available.

    Parameters:
    - cve_id (str): The CVE identifier (e.g., "CVE-2021-34527")
    - api_key (str): Optional API key for higher request rate limits

    Returns:
    - float or None: The CVSS base score, or None if not available
    """
    
    url = f"https://services.nvd.nist.gov/rest/json/cves/2.0?cveId={cve_id}"

    headers = {
        'User-Agent': 'CVE-Score-Lookup/1.0'
    }

    if api_key:
        headers['apiKey'] = api_key

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()

            # Proceed only if the CVE is found in the response
            if data.get('totalResults', 0) > 0:
                vuln = data['vulnerabilities'][0]['cve']
                metrics = vuln.get('metrics', {})

                # Try CVSS v3.1
                if 'cvssMetricV31' in metrics:
                    return metrics['cvssMetricV31'][0]['cvssData']['baseScore']
                # Fall back to CVSS v2 if v3 is not available
                elif 'cvssMetricV2' in metrics:
                    return metrics['cvssMetricV2'][0]['cvssData']['baseScore']
                else:
                    return None
            else:
                return None
        else:
            print(f"[{cve_id}] HTTP Error: {response.status_code}")
            return None

    except Exception as e:
        print(f"[{cve_id}] Error querying NVD API: {e}")
        return None

# Liste des CVE valides dans le DataFrame
cve_ids_missing_cvss = df['cve_id'][df['cve_id'].notna()].unique().tolist()

# Dictionnaire pour stocker les résultats
results = {}

# Ta clé API ici
api_key = "87665cf3-7426-4d20-af69-de8eaa8d4994"

# Boucle de récupération avec barre de progression et respect du quota de 100 requêtes/min
for cve_id in tqdm(cve_ids_missing_cvss, desc="Fetching CVSS from NVD"):
    cvss = get_cvss_from_nvd(cve_id, api_key)
    results[cve_id] = cvss
    time.sleep(0.6)  # 100 requests per minute = 1 request every 0.6 seconds

# Affichage (optionnel) d’un extrait des résultats
for k, v in list(results.items())[:10]:
    print(f"{k}: {v}")


Fetching CVSS from NVD:   5%|▌         | 64/1217 [01:43<30:57,  1.61s/it]

In [ ]:
def cvss(x):
    if pd.isna(x.cve_id):
        return x.cvss_score
    else:
        return results[x.cve_id]


df["cvss"] = df.apply(cvss, axis=1)


0.6

In [ ]:
def cvss(x):
    if pd.isna(x.cve_id):
        return x.cvss_score
    else:
        return results[x.cve_id]


df["cvss"] = df.apply(cvss, axis=1)


(1217,)

In [ ]:
df.cve_id[df.cve_id.notna() & df.cvss_score.isna()].tolist()

['CVE-2021-15352',
 'CVE-2022-9574',
 'CVE-2018-23699',
 'CVE-2019-22844',
 'CVE-2019-26136',
 'CVE-2018-21202',
 'CVE-2020-6203',
 'CVE-2019-7380',
 'CVE-2023-17220',
 'CVE-2022-19699',
 'CVE-2022-3631',
 'CVE-2020-4098',
 'CVE-2024-15311',
 'CVE-2018-18825',
 'CVE-2023-4143',
 'CVE-2020-7168',
 'CVE-2018-10069',
 'CVE-2018-1992',
 'CVE-2024-18409',
 'CVE-2018-15166',
 'CVE-2024-8836',
 'CVE-2018-17466',
 'CVE-2019-4477',
 'CVE-2023-17411',
 'CVE-2024-1299',
 'CVE-2021-6131',
 'CVE-2019-27173',
 'CVE-2021-29106',
 'CVE-2022-15750',
 'CVE-2018-10769',
 'CVE-2023-8594',
 'CVE-2022-1261',
 'CVE-2024-16271',
 'CVE-2023-8202',
 'CVE-2019-9372',
 'CVE-2021-12989',
 'CVE-2024-11036',
 'CVE-2022-6312',
 'CVE-2022-21514',
 'CVE-2021-25582',
 'CVE-2020-2531',
 'CVE-2018-4511',
 'CVE-2019-14730',
 'CVE-2020-22442',
 'CVE-2022-22964',
 'CVE-2023-4910',
 'CVE-2018-20225',
 'CVE-2023-16040',
 'CVE-2022-29278',
 'CVE-2019-23728',
 'CVE-2021-23691',
 'CVE-2024-19251',
 'CVE-2023-8275',
 'CVE-2020-181

In [ ]:
_ = df[["cve_id", "cvss_score", ]][df.cve_id.notna()]
_[_.cve_id.duplicated(keep=False)].sort_values("cve_id")

,cve_id,cvss_score
221,CVE-2018-10069,NaN
5134,CVE-2018-10069,NaN
4695,CVE-2018-11200,6.189107
5078,CVE-2018-11200,6.189107
2287,CVE-2018-20240,NaN
5068,CVE-2018-20240,NaN
2177,CVE-2018-20365,NaN
5191,CVE-2018-20365,NaN
5251,CVE-2018-20744,6.000000
4031,CVE-2018-20744,6.000000


In [ ]:
df[["cvss_score", "cve_id"]][(df.cvss_score.isna())  & (df.cve_id.notna())].shape

(563, 2)

In [1]:
pd.set_option('display.max_rows', 100)  # ou un nombre plus grand si tu veux
df[["severity", "cvss_score", "status"]][df.cvss_score.notna() & (~df.status.isin(["New", "In Progress"]))]

NameError: name 'pd' is not defined

In [ ]:
df[["cvss_score", "cve_id"]][(df.cvss_score.notna())  & (df.cve_id.notna()) & (df[["cvss_score", "cve_id"]].duplicated())]

,cvss_score,cve_id
5013,6.900000,CVE-2023-10454
5015,2.249344,CVE-2020-10461
5023,1.400000,CVE-2019-15493
5024,0.539518,CVE-2019-8544
5026,1.228191,CVE-2022-12199
5046,3.471037,CVE-2021-20750
5050,0.848486,CVE-2021-28091
5056,8.800000,CVE-2023-8672
5077,6.000000,CVE-2022-6386
5078,6.189107,CVE-2018-11200


In [ ]:
df.groupby('status')["cvss_score"].describe()

,count,mean,std,min,25%,50%,75%,max
status,,,,,,,,
False Positive,172.0,5.349251,2.663804,0.172216,3.100000,5.700000,7.500000,9.910113
In Progress,564.0,5.239771,2.639541,0.185735,2.900000,5.300000,7.500000,10.000000
New,890.0,5.302895,2.607924,0.169055,3.077705,5.249045,7.500000,10.000000
Resolved,163.0,5.130400,2.732187,0.187501,2.819951,5.100000,7.142677,10.000000


In [ ]:
df.groupby(['status',"severity"])["alert_id"].count()

status          severity
False Positive  Critical      46
                High         117
                Low          212
                Medium       141
In Progress     Critical     174
                High         303
                Low          647
                Medium       506
New             Critical     252
                High         541
                Low         1034
                Medium       759
Resolved        Critical      63
                High          91
                Low          218
                Medium       148
Name: alert_id, dtype: int64

In [ ]:
df.detection_tool.value_counts()

detection_tool
IDS                506
WAF                495
EDR                490
NIDS               488
Rustscan           484
Firewall           480
Qualys             479
Manual Analysis    476
Nuclei             457
SIEM               449
Nessus             448
Name: count, dtype: int64